<a href="https://colab.research.google.com/github/kevininhe/DeepLearning/blob/main/Proyecto_Final_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![image](https://docs.google.com/uc?export=download&id=1NUy1Q-abpoV9XYK9qT9t8Mdhj3ZVlveO)

# **Proyecto Deep Learning**
## **Detección de incendios con Redes Neuronales Convolucionales**
## **Integrantes**

* Fabián Ramirez
* David Vasquez
* Kevin Infante Hernández

## **Objetivos**
*   Utilizar un modelo U-Net pre-entrenado para la detección de incendios con imágenes satelitales de todo el mundo y especializarlo en incendios en Sur América usando fine-tuning.
*   Reproducir el código creado por Pereira et. al [1] para crear el modelo U-Net pre-entrenado usado en este proyecto, adaptarlo para ser ejecutado en Google Colab y realizar las modificaciones pertinentes para poder hacer fine-tuning de acuerdo a la documentación de Keras.
*   Comparar el desempeño en la detección de incendios en Sur América del modelo pre-entrenado por Pereira et. al y del modelo encontrado con fine-tuning usando las métricas de precisión, recall, Intersection Over Union (IoU), F-Score y el Jaccard score promedio.

## **Problema**

Los incendios activos son desastres naturales devastadores que causan daños socioeconómicos en todo el mundo, por lo cual su oportuna detección se ha convertido en una tarea muy importante para el planeta y la humanidad.

Las imágenes satelitales son un medio muy valioso para la detección de incendios dado su alcance global y a los datos que contienen. La gran cantidad de información que guardan y el volumen de imágenes satelitales existente hace de la detección de incendios en imágenes satelitales un problema complejo, por lo cual el uso de técnicas avanzadas como el Deep Learning podrían permitir obtener modelos que ayuden a identificar estos incendios con la mayor precisión posible. Sin embargo, varias dificultades impidieron utilizar técnicas de Deep Learning hasta hace poco.

Una de las dificultades principales era la falta de un set de datos anotado sobre el cual se pudiera realizar el entrenamiento de esta clase de modelos. Sin embargo, gracias al trabajo realizado en 2021 en Pereira et. al [1], donde los autores analizaron una gran cantidad de imágenes del satélite Landsat 8 en búsqueda de incendios activos y generaron las máscaras correspondientes que permiten identificar los píxeles con incendios, ahora se cuenta con un set con imágenes anotadas de incendios forestales en los 5 continentes que ha permitido el surgimiento de varias propuestas.

Sobre este set de datos mundial Pereira et. al realizaron el entrenamiento de varios modelos, sin embargo, en caso de que se necesite un modelo especializado en una región de interés en específico como la selva amazonica, es probable que se necesite un modelo más específico, pero sin perder el conocimiento adquirido gracias al entrenamiento con imágenes alrededor del mundo.

En este proyecto se plantea el uso de fine-tuning para mejorar un modelo pre-entrenado con imágenes de todo el mundo y especializarlo en una región en específico, mejorando las métricas de detección de incendios sobre esta región.


[1]: <https://www.sciencedirect.com/science/article/abs/pii/S092427162100160X?via%3Dihub>

### 0. Importación de paquetes

A continuación se importan las librerías utilizadas en para la creación de los modelos y su entrenamiento

In [ ]:
!pip install rasterio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 20.9 MB 1.3 MB/s 


In [ ]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import pandas as pd
import numpy as np
import rasterio
import sys
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from scipy.ndimage.measurements import label
import glob
import threading
from sklearn.utils import shuffle as shuffle_lists
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

Importar archivos desde Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Entendimiento del Negocio

Alrededor de la tierra orbitan varios tipos de satélites, algunos de ellos capturan constantemente imágenes de la tierra que sirven para múltiples propósitos. Las imágenes satelitales son particularmente valiosas ya que capturan un espectro de ondas mucho mayor al visible por el ser humano, permitiendo obtener información tan detallada como la temperatura del suelo, el estado de la vegetación, entre otros [2].

Uno de estos satélites es el Landsat 8, el cual, además de capturar el espectro de ondas visible para el ser humano, captura ondas infrarrojas térmicas, infrarrojas de onda corta, entre otras, que juntas conforman una imagen con un total de 11 canales (una imagen a color solo tiene 3 canales: rojo, verde y azul). Un píxel de estas imágenes corresponde aproximadamente a 30 m2 (dependiendo del canal), y cada imágen cubre aproximadamente un área de 185 x 180 kilómetros [3].

Toda esta información hace que las imágenes satelitales sean una fuente de información valiosa para la detección de incendios alrededor del mundo. De hecho, antes del uso de Deep Learning, ya se utilizaban técnicas basadas en operaciones matemáticas sobre las diferentes bandas de las imágenes satelitales para la detección de incendios. Sin embargo, dado el gran volumen de imágenes satelitales que existen y la variedad de condiciones en las que se puede generar un incendio, se hace necesario el uso de Deep Learning para esta tarea.

Sin embargo, tal como se mencionó previamente, una de las dificultades principales que evitaban la creación de modelos de Deep Learning para el cumplimiento de esta tarea era que no se disponía de un set de datos anotado que permitiera el entrenamiento de modelos de Deep Learning, dificultad que fue afrontada en Pereira et. al y que permitió tener un conjunto de imágenes satelitales de los 5 continentes anotadas y listas para ser usadas en el entrenamiento de modelos de Deep Learning.

Las imágenes fueron tomadas en Agosto de 2019, temporada del pico de mayores incendios a nivel global.

[2]: <https://landsat.gsfc.nasa.gov/satellites/landsat-8/landsat-8-bands/>
[3]: <https://www.mdpi.com/2072-4292/6/11/10286>

### 2. Entendimiento de los Datos

Para simplificar el entrenamiento de los modelos de Deep Learning, Pereira et. al dividieron las imágenes capturadas del satélite Landsat 8 en parches de 256 x 256 píxeles, y, eliminaron la banda 8 (pancromática) de estas imágenes.

Una vez hecha esta segmentación utilizaron tres técnicas diferentes ([Kumar y Roy (2018)](https://www.tandfonline.com/doi/full/10.1080/17538947.2017.1391341), [Murphy et. al (2016)](https://linkinghub.elsevier.com/retrieve/pii/S0034425716300554) y [Schroeder et. al (2016)](https://www.sciencedirect.com/science/article/pii/S0034425715301206?via%3Dihub)), su intercepto y un sistema de votación (es decir, 5 técnicas diferentes) para encontrar los píxeles con incendios y hacer así la anotación de las imágenes. La anotación de las imágenes es una máscara de 256 x 256 cuyos píxeles tienen valores de 1 (tiene incendio) o 0 (no tiene incendio).

Por ende, el set de datos consiste de las imágenes satelitales segmentadas, sus máscaras correspondientes generadas por las 5 técnicas que decidieron usar (es decir, para cada imágen se generaron 5 máscaras) y una metadata de las imágenes que tiene información adicional sobre la información capturada. En total el set de datos tiene un tamaño de aproximadamente 165 GBs de información, distribuidos de la siguiente manera [4]

![image](https://github.com/pereira-gha/activefire/blob/main/map_downloads.png?raw=true)

[4]: <https://github.com/pereira-gha/activefire>

### 3. Preparación de datos

Dado que en Pereira et. al ya se preprocesaron los imágenes, en la preparación de datos se hizo lo siguiente:


1.   Descarga de imágenes correspondientes a Sur América junto con sus máscaras. La técnica de generación de máscaras seleccionada fue "Intersección", que es la intersección de los tres métodos listados en la sección anterior.
2.   División de las imágenes y máscaras de intersección descargadas en sets de entrenamiento, validación y test.
3.   Carga de imágenes y máscaras en Google Drive.

Usando las utilidades construidas por Pereira et. al se realizó la división de las imágenes y máscaras descargadas en sets de entrenamiento, validación y test, división representada en seis archivos planos: tres que especifican las imágenes de entrenamiento, validación, y pruebas, y tres que especifican las máscaras de entrenamiento, validación y pruebas.

Las siguientes son las rutas donde se guardaron las imágenes, las máscaras, y los archivos txt con la división de las imágenes:


*   **Imágenes**: '/content/drive/MyDrive/DeepLearning/dataset/images/patches/'
*   **Máscaras**: '/content/drive/MyDrive/DeepLearning/dataset/masks/intersection/'
*   **Archivos planos**: /content/drive/MyDrive/DeepLearning/datasetDef


Las imágenes y máscaras de entrenamiento y validación se utilizaran para hacer el fine-tuning del modelo pre-entrenado por Pereira et.al, mientras que las imágenes de test se utilizarán para evaluar la detección de incendios en imágenes de Sur América tanto del modelo pre-entrenado como del modelo encontrado usando fine-tuning.



### 4. Modelamiento

Primero se definen las constantes, entre las cuales se encuentran el número de filtros de las capas convolucionales, el número de canales que se tomarán de la imágen satelital, y el tamaño de la imágen de entrada.

Además, se define la ruta del archivo de pesos, el cual almacena los pesos del modelo pre-entrenado por Pereira et. al.

In [ ]:
N_FILTERS = 64
N_CHANNELS = 10

IMAGE_SIZE = (256, 256)
WEIGHTS_FILE = "/content/drive/MyDrive/DeepLearning/weights/model_unet_Intersection_final_weights.h5"

A continuación se realiza la definición de la arquitectura U-Net usada en este proyecto, que es casi la misma que en Pereira et. al pero con una modificación para que se pueda hacer fine-tuning con ella.

La modificación consiste en dejar el parámetro `trainable=False` en las capas de Batch Normalization para que para que estas mantengan sus pesos fijos durante el fine-tuning ya que los pesos de las capas de batch-normalization no deberían cambiar en el fine-tuning.

Fuente: 
- [Documentación Keras](https://keras.io/guides/transfer_learning/)
- [Pregunta Stack Overflow](https://stackoverflow.com/a/59525939/3998212)

In [ ]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization(trainable=False)(x)
    x = Activation("relu")(x)
    
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization(trainable=False)(x)
    x = Activation("relu")(x)
    return x

In [ ]:
def get_unet(nClasses, input_height=256, input_width=256, n_filters = 16, dropout = 0.1, batchnorm = True, n_channels=10):
    input_img = Input(shape=(input_height,input_width, n_channels))

    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
def get_model(nClasses=1, input_height=128, input_width=128, n_filters = 16, dropout = 0.1, batchnorm = True, n_channels=10):
    model = get_unet

    return model(
            nClasses      = nClasses,  
            input_height  = input_height, 
            input_width   = input_width,
            n_filters     = n_filters,
            dropout       = dropout,
            batchnorm     = batchnorm,
            n_channels    = n_channels
        )

#### Cargue de pesos en el modelo

En Pereira et. al definen por aparte la arquitectura del modelo y sus pesos. Una vez definida la arquitectura se le cargan sus pesos pre-entrenados, los cuales se encuentran en un archivo h5.

In [ ]:
model = get_model(input_height=IMAGE_SIZE[0], input_width=IMAGE_SIZE[1], n_filters=N_FILTERS, n_channels=N_CHANNELS)

print('Loading weights...')
model.load_weights(WEIGHTS_FILE)
print('Weights Loaded')

model.summary()

Loading weights...
Weights Loaded
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  5824        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                           

### 5. Evaluación del modelo pre-entrenado

Para realizar la evaluación del modelo pre-entrenado por Pereira et. al en imágenes de Sur América se deben hacer dos pasos:
1. Generar un archivo plano tanto de la anotación de la imágen (su máscara) como de la predicción del modelo. Este archivo plano cuenta con 256 filas y 256 columnas con valores de 1 o 0, 1 indica que hay un incendio en el píxel y 0 indica la ausencia de incendio.
2. Se calculan las métricas de similitud comparando los archivos generados

#### Generación archivo plano

Se definen las constantes, entre las que se encuentran la ruta de las imágenes y sus anotaciones (máscaras), los archivos planos (csv) que indican la división de las imágenes en set de entrenamiento, pruebas y validación, el nombre y ruta de los archivos planos generados, el método usado para calcular la máscara (intersection), y el `TH_FIRE` (threshold fire), que es el valor (luego de ser normalizado) que debe alcanzar un píxel en la predicción del modelo para que sea considerado como un píxel con incendio.

Finalmente, se usa una constante de valor máximo del píxel para normalizar la imagen.

Nótese que se están convirtiendo a archivos planos solamente las imágenes y máscaras de test, ya que sobre estas se van a calcular las métricas de rendimiento del modelo.

In [ ]:
IMAGES_PATH = '/content/drive/MyDrive/DeepLearning/dataset/images/patches/'
MASKS_PATH = '/content/drive/MyDrive/DeepLearning/dataset/masks/intersection/'

IMAGES_CSV = '/content/drive/MyDrive/DeepLearning/datasetDef/images_test.csv'
MASKS_CSV = '/content/drive/MyDrive/DeepLearning/datasetDef/masks_test.csv'

OUTPUT_DIR = '/content/drive/MyDrive/DeepLearning/log'
OUTPUT_CSV_FILE = 'output_v1_{}_{}.csv'.format('unet', 'Intersection')
WRITE_OUTPUT = True

MASK_ALGORITHM = 'intersection'
TH_FIRE = 0.25
MAX_PIXEL_VALUE = 65535 # Max. pixel value, used to normalize the image

Métodos auxiliares para convertir una imágen satelital en arreglo de Numpy. El get_img_arr normaliza los valores de los pixeles dividiendolos por el valor máximo que puede alcanzar un píxel en una imágen satelital de Landsat 8

In [ ]:
def get_img_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))    
    img = np.float32(img)/MAX_PIXEL_VALUE
    
    return img

def get_mask_arr(path):
    img = rasterio.open(path).read().transpose((1, 2, 0))
    seg = np.float32(img)
    return seg

Método para convertir las imágenes y las máscaras de test en archivos planos.

**Nota**: Se agregó un if en el método ya que Colab detuvo el runtime luego de que se habían procesado 2710 imágenes. Este if permitió saltarse las imágenes que ya habían sido generadas.  

In [ ]:
def imageToTextFile():
  print('Loading images...')

  images_df = pd.read_csv(IMAGES_CSV)
  masks_df = pd.read_csv(MASKS_CSV)

  images = []
  masks = []

  images = [ os.path.join(IMAGES_PATH, image) for image in images_df['images'] ]
  masks = [ os.path.join(MASKS_PATH, mask) for mask in masks_df['masks'] ]

  print('# of Images: {}'.format( len(images)) )
  print('# of Masks: {}'.format( len(masks)) )

  step = 0
  steps = len(images)
  for image, mask in zip(images, masks):

    # Se agregó este if ya que el runtime de Colab se detuvo en medio de la ejecución
    if step>2710:
      try:
          img = get_img_arr(image)
          # img = get_img_762bands(image)
          
          mask_name = os.path.splitext(os.path.basename(mask))[0]
          image_name = os.path.splitext(os.path.basename(image))[0]

          if mask_name.replace('_{}'.format(MASK_ALGORITHM), '') != image_name:
              print('[ERROR] Dont match {} - {}'.format(mask_name, image_name))
              sys.exit()

          mask = get_mask_arr(mask)

          txt_mask_path = os.path.join(OUTPUT_DIR, 'arrays', 'grd_' + mask_name + '.txt') 
          txt_pred_path = os.path.join(OUTPUT_DIR, 'arrays', 'det_' + image_name + '.txt') 

          y_pred = model.predict(np.array( [img] ), batch_size=1)

          y_true = mask[:,:,0] > TH_FIRE
          y_pred = y_pred[0, :, :, 0] > TH_FIRE

          np.savetxt(txt_mask_path, y_true.astype(int), fmt='%i')
          np.savetxt(txt_pred_path, y_pred.astype(int), fmt='%i')

          step += 1
          
          if step%100 == 0:
              print('Step {} of {}'.format(step, steps)) 
              
      except Exception as e:
          print(e)
          
          with open(os.path.join(OUTPUT_DIR, "error_log_inference.txt"), "a+") as myfile:
              myfile.write(str(e))
    else:
      step+=1
      print(step)


Ejecución de la generación de archivos planos. Se tienen 2849 parches de test. Nótese que, ya que las primeras 2710 imágenes ya habían sido generadas en una ejecución anterior, el log se salta hasta la imágen 2711, y a partir de ahí hace la conversión.

In [ ]:
with tf.device('/device:GPU:0'):
  imageToTextFile()

Loading images...
# of Images: 2849
# of Masks: 2849
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264

Se revisan los archivos creados. Unos corresponden a la máscara de entrenamiento (grd) y los otros corresponden a la predicción del modelo (det). Es decir, por cada imágen se crean dos archivos txt

In [ ]:
!ls /content/drive/MyDrive/DeepLearning/log/arrays

det_LC08_L1GT_003062_20200810_20200810_01_RT_p00487.txt
det_LC08_L1GT_008064_20200813_20200813_01_RT_p00462.txt
det_LC08_L1GT_008064_20200813_20200813_01_RT_p00524.txt
det_LC08_L1GT_010057_20200827_20200827_01_RT_p00673.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00461.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00491.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00701.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00729.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00530.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00599.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00718.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00784.txt
det_LC08_L1GT_227080_20200811_20200811_01_RT_p00209.txt
det_LC08_L1GT_227080_20200811_20200811_01_RT_p00680.txt
det_LC08_L1GT_230055_20200816_20200816_01_RT_p00376.txt
det_LC08_L1GT_230056_20200816_20200816_01_RT_p00351.txt
det_LC08_L1TP_001054_20200828_20200828_01_RT_p00424.txt
det_LC08_L1TP_001054_20200828_20200828_01_RT_p00

Se muestra el contenido del primer archivo creado

In [ ]:
arraysPath = os.path.join(OUTPUT_DIR,'arrays')
fileInPath = os.listdir(arraysPath)[0]
fullPath = os.path.join(arraysPath,fileInPath)
with open(fullPath, "r") as file:
  print(file.read())

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 

#### Cálculo métricas de similitud

Métodos para el cálculo de diferentes métricas

In [ ]:
def statistics (y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return tn, fp, fn, tp

def statistics2 (y_true, y_pred):
    py_actu = pd.Series(y_true, name='Actual')
    py_pred = pd.Series(y_pred, name='Predicted')
    df_confusion = pd.crosstab(py_actu, py_pred)
    return df_confusion[0][0], df_confusion[0][1], df_confusion[1][0], df_confusion[1][1]

def statistics3 (y_true, y_pred):
    y_pred_neg = 1 - y_pred
    y_expected_neg = 1 - y_true

    tp = np.sum(y_pred * y_true)
    tn = np.sum(y_pred_neg * y_expected_neg)
    fp = np.sum(y_pred * y_expected_neg)
    fn = np.sum(y_pred_neg * y_true)
    return tn, fp, fn, tp

def jaccard3 (im1, im2):
    """
    Computes the Jaccard metric, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    jaccard : float
        Jaccard metric returned is a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
    
    Notes
    -----
    The order of inputs for `jaccard` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """

    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    intersection = np.logical_and(im1, im2)

    union = np.logical_or(im1, im2)

    jaccard_fire = intersection.sum() / float(union.sum())

    im1 = np.logical_not(im1)
    im2 = np.logical_not(im2)

    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    intersection = np.logical_and(im1, im2)

    union = np.logical_or(im1, im2)

    jaccard_non_fire = intersection.sum() / float(union.sum())

    jaccard_avg = (jaccard_fire + jaccard_non_fire)/2

    return jaccard_fire, jaccard_non_fire, jaccard_avg

def pixel_accuracy (y_true, y_pred):
    sum_n = np.sum(np.logical_and(y_pred, y_true))
    sum_t = np.sum(y_true)
 
    if (sum_t == 0):
        pixel_accuracy = 0
    else:
        pixel_accuracy = sum_n / sum_t
    return pixel_accuracy    

def connected_components (array):
   structure = np.ones((3, 3), dtype=np.int)  # 8-neighboorhood
   labeled, ncomponents = label(array, structure)
   return labeled

def region_relabel (y_true, y_pred):
   index = 0
   for p in y_true:
      if y_pred[index] == 1:
          if y_true[index] > 0:
              y_pred[index] = y_true[index]
          else:
              y_pred[index] = 999
      index += 1

In [ ]:
def getScores():
  y_pred_all_v1 = []
  y_true_all_v1 = []
  y_pred_all_multi_v1 = []
  y_true_all_multi_v1 = []

  jaccard_score_sum_v1 = 0
  f1_score_sum_v1 = 0
  pixel_accuracy_sum_v1 = 0

  nsum_v1 = 0
  step = 0
          
  txts_mask_path = sorted(glob.glob(os.path.join(OUTPUT_DIR, 'arrays', 'grd_*.txt')))
  txts_pred_path = sorted(glob.glob(os.path.join(OUTPUT_DIR, 'arrays', 'det_*.txt')))

  print('# Masks: {}'.format(len(txts_mask_path)))
  print('# Pred.: {}'.format(len(txts_pred_path)))

  steps = len(txts_mask_path)
      
  for txt_mask_path, txt_pred_path in zip(txts_mask_path, txts_pred_path):
      
      try:
          if txt_mask_path.replace('grd', 'det').replace('_{}'.format(MASK_ALGORITHM), '') != txt_pred_path:
              print('[ERROR] Dont match {} - {}'.format(txt_mask_path, txt_pred_path))
              sys.exit()

          y_true = np.loadtxt(txt_mask_path, usecols=range(IMAGE_SIZE[1]))
          y_pred = np.loadtxt(txt_pred_path, usecols=range(IMAGE_SIZE[1]))

          y_true = np.array(y_true, dtype=np.uint8)
          y_pred = np.array(y_pred, dtype=np.uint8)

          y_pred = y_pred.flatten()
          y_true = y_true.flatten()

          y_pred_all_v1.append(y_pred)
          y_true_all_v1.append(y_true)

          jaccard_score_v1 = jaccard_score(y_true, y_pred, average='macro')
          jaccard_score_sum_v1 = jaccard_score_sum_v1 + jaccard_score_v1

          f1_score_v1 = f1_score(y_true, y_pred)
          f1_score_sum_v1 = f1_score_sum_v1 + f1_score_v1

          pixel_accuracy_v1 = pixel_accuracy(y_true, y_pred)
          pixel_accuracy_sum_v1 = pixel_accuracy_sum_v1 + pixel_accuracy_v1

          nsum_v1 = nsum_v1 + 1

          count_fire_pixel_mask = np.sum(y_true)
          count_fire_pixel_pred = np.sum(y_pred)
          
          step += 1
          if step%100 == 0:
              print('Step {} of {}'.format(step, steps)) 
              
      except Exception as e:
          print(e)
          
          with open(os.path.join(OUTPUT_DIR, "error_log.txt"), "a+") as myfile:
              myfile.write(str(e))
      


  y_pred_all_v1 = np.array(y_pred_all_v1, dtype=np.uint8)
  y_pred_all_v1 = y_pred_all_v1.flatten()

  y_true_all_v1 = np.array(y_true_all_v1, dtype=np.uint8)
  y_true_all_v1 = y_true_all_v1.flatten()

  tn, fp, fn, tp = statistics3 (y_true_all_v1, y_pred_all_v1)

  P = float(tp)/(tp + fp)
  R = float(tp)/(tp + fn)
  IoU = float(tp)/(tp+fp+fn)
  F = (2 * P * R)/(P + R)
  print('P: :', P, ' R: ', R, ' IoU: ', IoU, ' F-score: ', F)
  print('Jaccard score average', jaccard_score_sum_v1/nsum_v1)

In [ ]:
with tf.device('/device:GPU:0'):
  getScores()

# Masks: 2849
# Pred.: 2849
Step 100 of 2849
Step 200 of 2849
Step 300 of 2849
Step 400 of 2849
Step 500 of 2849
Step 600 of 2849
Step 700 of 2849
Step 800 of 2849
Step 900 of 2849
Step 1000 of 2849
Step 1100 of 2849
Step 1200 of 2849
Step 1300 of 2849
Step 1400 of 2849
Step 1500 of 2849
Step 1600 of 2849
Step 1700 of 2849
Step 1800 of 2849
Step 1900 of 2849
Step 2000 of 2849
Step 2100 of 2849
Step 2200 of 2849
Step 2300 of 2849
Step 2400 of 2849
Step 2500 of 2849
Step 2600 of 2849
Step 2700 of 2849
Step 2800 of 2849
P: : 0.8749501323758749  R:  0.9982207878186031  IoU:  0.873587775202781  F-score:  0.9325293287721537
Jaccard score average 0.9463829670347707


### 6. Fine Tuning

Una vez obtenidas las métricas de desempeño del modelo pre-entrenado sobre el set de test de imágenes de Sur América, se procede a realizar el fine-tuning del modelo, luego del cual se evaluará su desempeño y se compararan sus métricas de desempeño contra las del modelo pre-entrenado original usando el mismo set de test de Sur América.

Se definen las constantes, entre las cuales destacan el `CHECKPOINT_PERIOD`, que indica cada cuantas épocas se deben guardar los pesos del modelo en formato h5, el `EARLY_STOP_PATIENCE`, usado en el callback de parada temprana, y el número de épocas, establecido como 5 para evitar el overfitting del modelo.

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/DeepLearning/train_output/'
BATCH_SIZE = 16
RANDOM_STATE = 42
EARLY_STOP_PATIENCE = 3
CHECKPOINT_PERIOD = 1
CHECKPOINT_MODEL_NAME = 'checkpoint-{}-{}-epoch_{{epoch:02d}}.hdf5'.format('unet', MASK_ALGORITHM)
# If not zero will be load as weights
INITIAL_EPOCH = 0
RESTART_FROM_CHECKPOINT = None
if INITIAL_EPOCH > 0:
    RESTART_FROM_CHECKPOINT = os.path.join(OUTPUT_DIR, 'checkpoint-{}-{}-epoch_{:02d}.hdf5'.format('unet', MASK_ALGORITHM, INITIAL_EPOCH))
FINAL_WEIGHTS_OUTPUT = 'model_{}_{}_final_weights.h5'.format('unet', MASK_ALGORITHM)
EPOCHS = 5
WORKERS = 4
PLOT_HISTORY = True

Se definen los sets de entrenamiento y validación, que son los archivos planos (csv) que definen la división de las imágenes y sus máscaras en sets de entrenamiento, validación y test.

In [ ]:
x_train = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasetDef/images_train.csv')
y_train = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasetDef/masks_train.csv')
x_val = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasetDef/images_val.csv')
y_val = pd.read_csv('/content/drive/MyDrive/DeepLearning/datasetDef/masks_val.csv')

Métodos para que las listas sean iteradores, de modo que no se sature la RAM

In [ ]:
class threadsafe_iter:
    """
    Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    From: https://github.com/keras-team/keras/issues/1638#issuecomment-338218517
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        with self.lock:
            return self.it.__next__()

def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe."""
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))

    return g

@threadsafe_generator
def generator_from_lists(images_path, masks_path, batch_size=32, shuffle = True, random_state=None):
   
    images = []
    masks = []

    i = 0 # used to shuffle samples
    while True:
        
        if shuffle:
            if random_state is None:
                images_path, masks_path = shuffle_lists(images_path, masks_path)
            else:
                images_path, masks_path = shuffle_lists(images_path, masks_path, random_state= random_state + i)
                i += 1 # keep a consistent order in shuffle


        for img_path, mask_path in zip(images_path, masks_path):

            img = get_img_arr(img_path)
            mask = get_mask_arr(mask_path)
            images.append(img)
            masks.append(mask)

            if len(images) >= batch_size:
                yield (np.array(images), np.array(masks))
                images = []
                masks = []


Método para graficar la historia de entrenamiento

In [ ]:
def plot_history(history, out_dir):

    plt.plot(history.history['acc'], label='training')
    plt.plot(history.history['val_acc'], label='validation')
    plt.legend()
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.savefig(os.path.join(out_dir, "acc.png"), dpi=300, bbox_inches='tight')
    plt.clf()
  
    plt.plot(history.history['loss'], label='training')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    plt.grid()
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.savefig(os.path.join(out_dir, "loss.png"), dpi=300, bbox_inches='tight')
    plt.clf()

Se definen los iteradores y se re-compila el modelo con una tasa de entrenamiento muy baja (`1e-5`), definida dentro del constructor del optimizador (Adam), esto para que en el fine-tuning los pesos pre-entrenados no varíen en gran medida.

In [ ]:
# Map the images and mask path
images_train = [ os.path.join(IMAGES_PATH, image) for image in x_train['images'] ]
masks_train = [ os.path.join(MASKS_PATH, mask) for mask in y_train['masks'] ]

images_validation = [ os.path.join(IMAGES_PATH, image) for image in x_val['images'] ]
masks_validation = [ os.path.join(MASKS_PATH, mask) for mask in y_val['masks'] ]

train_generator = generator_from_lists(images_train, masks_train, batch_size=BATCH_SIZE, random_state=RANDOM_STATE)
validation_generator = generator_from_lists(images_validation, masks_validation, batch_size=BATCH_SIZE, random_state=RANDOM_STATE)

# Se compila el modelo con un learning rate muy bajo para fine tuning
model.compile(optimizer = Adam(1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  5824        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

#### Entrenamiento del modelo

Definición de callbacks y entrenamiento del modelo.

**Nota**: Se tuvo que detener el entrenamiento del modelo en la tercera época y tomar el archivo h5 generado en esta época ya que los recursos computacionales asignados para el mes en Colab Pro estaban por agotarse, y para evitar cualquier comportamiento inesperado se optó por detener el fine-tuning.

In [ ]:
from pickle import NONE
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=EARLY_STOP_PATIENCE)
checkpoint = ModelCheckpoint(os.path.join(OUTPUT_DIR, CHECKPOINT_MODEL_NAME), monitor='loss', verbose=1,
                             save_best_only=True, mode='auto', period=CHECKPOINT_PERIOD)

if INITIAL_EPOCH > 0:
    model.load_weights(RESTART_FROM_CHECKPOINT)

print('Training using {}...'.format(MASK_ALGORITHM))
history = None
with tf.device('/device:GPU:0'):
  history = model.fit_generator(
      train_generator,
      steps_per_epoch=len(images_train) // BATCH_SIZE,
      validation_data=validation_generator,
      validation_steps=len(images_validation) // BATCH_SIZE,
      callbacks=[checkpoint, es],
      epochs=EPOCHS,
      workers=WORKERS,
      initial_epoch=INITIAL_EPOCH
  )
  print('Train finished!')


print('Saving weights')
model_weights_output = os.path.join(OUTPUT_DIR, FINAL_WEIGHTS_OUTPUT)
model.save_weights(model_weights_output)
print("Weights Saved: {}".format(model_weights_output))


if PLOT_HISTORY:
    plot_history(history, OUTPUT_DIR)

<ipython-input-28-8830520ececa>:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Training using intersection...
Epoch 1/5
142/142 [==============================] - ETA: 0s - loss: 3.0982e-05 - accuracy: 1.0000 
Epoch 1: loss improved from inf to 0.00003, saving model to /content/drive/MyDrive/DeepLearning/train_output/checkpoint-unet-intersection-epoch_01.hdf5
142/142 [==============================] - 4167s 29s/step - loss: 3.0982e-05 - accuracy: 1.0000 - val_loss: 2.6097e-05 - val_accuracy: 1.0000
Epoch 2/5
142/142 [==============================] - ETA: 0s - loss: 2.8557e-05 - accuracy: 1.0000 
Epoch 2: loss improved from 0.00003 to 0.00003, saving model to /content/drive/MyDrive/DeepLearning/train_output/checkpoint-unet-intersection-epoch_02.hdf5
142/142 [==============================] - 10105s 72s/step - loss: 2.8557e-05 - accuracy: 1.0000 - val_loss: 2.4604e-05 - val_accuracy: 1.0000
Epoch 3/5
142/142 [==============================] - ETA: 0s - loss: 2.7834e-05 - accuracy: 1.0000 
Epoch 3: loss improved from 0.00003 to 0.00003, saving model to /content/dri

KeyboardInterrupt: ignored

### Evaluación del modelo encontrado usando fine-tuning

Al igual que en la evaluación del modelo pre-entrenado se deben hacer dos pasos:
1. Generar un archivo plano tanto de la predicción del modelo como de la anotación de la imágen (máscara). Este archivo plano cuenta con 256 filas y 256 columnas con valores de 1 o 0.
2. Se calculan las métricas de similitud comparando los archivos generados a partir de la anotación de la imágen y de la predicción del modelo.

Se especifica la ruta del archivo de pesos generado en el fine-tuning

In [ ]:
WEIGHTS_FILE_FINAL = '/content/drive/MyDrive/DeepLearning/train_output/checkpoint-unet-intersection-epoch_03.hdf5'

Se cargan los nuevos pesos al modelo

In [ ]:
model = get_model(input_height=IMAGE_SIZE[0], input_width=IMAGE_SIZE[1], n_filters=N_FILTERS, n_channels=N_CHANNELS)

print('Loading weights...')
model.load_weights(WEIGHTS_FILE_FINAL)
print('Weights Loaded')

model.summary()

Loading weights...
Weights Loaded
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                0)]                                                               
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  5824        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 256, 64  256        ['conv2d[0][0]']                 
 alization)                     )                           

#### Generación archivo plano

Se definen nuevamente las constantes para generar el archivo plano

In [ ]:
IMAGES_PATH = '/content/drive/MyDrive/DeepLearning/dataset/images/patches/'
MASKS_PATH = '/content/drive/MyDrive/DeepLearning/dataset/masks/intersection/'

IMAGES_CSV = '/content/drive/MyDrive/DeepLearning/datasetDef/images_test.csv'
MASKS_CSV = '/content/drive/MyDrive/DeepLearning/datasetDef/masks_test.csv'

OUTPUT_DIR = '/content/drive/MyDrive/DeepLearning/log'
OUTPUT_CSV_FILE = 'output_v1_{}_{}.csv'.format('unet', 'Intersection')
WRITE_OUTPUT = True

MASK_ALGORITHM = 'intersection'
TH_FIRE = 0.25
MAX_PIXEL_VALUE = 65535 # Max. pixel value, used to normalize the image

Método de generación del archivo plano

In [ ]:
def imageToTextFile():
  print('Loading images...')

  images_df = pd.read_csv(IMAGES_CSV)
  masks_df = pd.read_csv(MASKS_CSV)

  images = []
  masks = []

  images = [ os.path.join(IMAGES_PATH, image) for image in images_df['images'] ]
  masks = [ os.path.join(MASKS_PATH, mask) for mask in masks_df['masks'] ]

  print('# of Images: {}'.format( len(images)) )
  print('# of Masks: {}'.format( len(masks)) )

  step = 0
  steps = len(images)
  for image, mask in zip(images, masks):
      try:
          img = get_img_arr(image)
          
          mask_name = os.path.splitext(os.path.basename(mask))[0]
          image_name = os.path.splitext(os.path.basename(image))[0]

          if mask_name.replace('_{}'.format(MASK_ALGORITHM), '') != image_name:
              print('[ERROR] Dont match {} - {}'.format(mask_name, image_name))
              sys.exit()

          mask = get_mask_arr(mask)

          txt_mask_path = os.path.join(OUTPUT_DIR, 'arraysNM', 'grd_' + mask_name + '.txt') 
          txt_pred_path = os.path.join(OUTPUT_DIR, 'arraysNM', 'det_' + image_name + '.txt') 

          y_pred = model.predict(np.array( [img] ), batch_size=1)

          y_true = mask[:,:,0] > TH_FIRE
          y_pred = y_pred[0, :, :, 0] > TH_FIRE

          np.savetxt(txt_mask_path, y_true.astype(int), fmt='%i')
          np.savetxt(txt_pred_path, y_pred.astype(int), fmt='%i')

          step += 1
          
          if step%100 == 0:
              print('Step {} of {}'.format(step, steps)) 
              
      except Exception as e:
          print(e)
          
          with open(os.path.join(OUTPUT_DIR, "error_log_inference.txt"), "a+") as myfile:
              myfile.write(str(e))


Creación de los archivos planos

In [ ]:
with tf.device('/device:GPU:0'):
  imageToTextFile()

Loading images...
# of Images: 2849
# of Masks: 2849
1/1 [==============================] - 0s 20ms/step
Step 100 of 2849
1/1 [==============================] - 0s 21ms/step
Step 200 of 2849
1/1 [==============================] - 0s 20ms/step
Step 300 of 2849
1/1 [==============================] - 0s 23ms/step
Step 400 of 2849
1/1 [==============================] - 0s 20ms/step
Step 500 of 2849
1/1 [==============================] - 0s 19ms/step
Step 600 of 2849
1/1 [==============================] - 0s 19ms/step
Step 700 of 2849
1/1 [==============================] - 0s 21ms/step
Step 800 of 2849
1/1 [==============================] - 0s 20ms/step
Step 900 of 2849
1/1 [==============================] - 0s 20ms/step
Step 1000 of 2849
1/1 [==============================] - 0s 20ms/step
Step 1100 of 2849
1/1 [==============================] - 0s 20ms/step
Step 1200 of 2849
1/1 [==============================] - 0s 20ms/step
Step 1300 of 2849
1/1 [==============================] - 0s 20ms

Se verifican que los archivos planos correspondientes a la predicción hayan sido creados

In [ ]:
!ls /content/drive/MyDrive/DeepLearning/log/arraysNM

det_LC08_L1GT_003062_20200810_20200810_01_RT_p00487.txt
det_LC08_L1GT_008064_20200813_20200813_01_RT_p00462.txt
det_LC08_L1GT_008064_20200813_20200813_01_RT_p00524.txt
det_LC08_L1GT_010057_20200827_20200827_01_RT_p00673.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00461.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00491.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00701.txt
det_LC08_L1GT_010059_20200827_20200827_01_RT_p00729.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00530.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00599.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00718.txt
det_LC08_L1GT_227079_20200811_20200811_01_RT_p00784.txt
det_LC08_L1GT_227080_20200811_20200811_01_RT_p00209.txt
det_LC08_L1GT_227080_20200811_20200811_01_RT_p00680.txt
det_LC08_L1GT_230055_20200816_20200816_01_RT_p00376.txt
det_LC08_L1GT_230056_20200816_20200816_01_RT_p00351.txt
det_LC08_L1TP_001054_20200828_20200828_01_RT_p00424.txt
det_LC08_L1TP_001054_20200828_20200828_01_RT_p00

Se verifica el primer archivo de la carpeta donde se guardaron los planos de la predicción

In [ ]:
arraysPath = os.path.join(OUTPUT_DIR,'arraysNM')
fileInPath = os.listdir(arraysPath)[0]
fullPath = os.path.join(arraysPath,fileInPath)
with open(fullPath, "r") as file:
  print(file.read())

0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

#### Cálculo de métricas de similitud

Se calculan las métricas de similitud usando los archivos planos de las máscaras y los archivos planos generados con las predicciones del modelo encontrado con fine-tuning

In [ ]:
def getScores():
  y_pred_all_v1 = []
  y_true_all_v1 = []
  y_pred_all_multi_v1 = []
  y_true_all_multi_v1 = []

  jaccard_score_sum_v1 = 0
  f1_score_sum_v1 = 0
  pixel_accuracy_sum_v1 = 0

  nsum_v1 = 0
  step = 0
          
  txts_mask_path = sorted(glob.glob(os.path.join(OUTPUT_DIR, 'arraysNM', 'grd_*.txt')))
  txts_pred_path = sorted(glob.glob(os.path.join(OUTPUT_DIR, 'arraysNM', 'det_*.txt')))

  print('# Masks: {}'.format(len(txts_mask_path)))
  print('# Pred.: {}'.format(len(txts_pred_path)))

  steps = len(txts_mask_path)
      
  for txt_mask_path, txt_pred_path in zip(txts_mask_path, txts_pred_path):
      
      try:
          if txt_mask_path.replace('grd', 'det').replace('_{}'.format(MASK_ALGORITHM), '') != txt_pred_path:
              print('[ERROR] Dont match {} - {}'.format(txt_mask_path, txt_pred_path))
              sys.exit()

          y_true = np.loadtxt(txt_mask_path, usecols=range(IMAGE_SIZE[1]))
          y_pred = np.loadtxt(txt_pred_path, usecols=range(IMAGE_SIZE[1]))

          y_true = np.array(y_true, dtype=np.uint8)
          y_pred = np.array(y_pred, dtype=np.uint8)

          y_pred = y_pred.flatten()
          y_true = y_true.flatten()

          y_pred_all_v1.append(y_pred)
          y_true_all_v1.append(y_true)

          jaccard_score_v1 = jaccard_score(y_true, y_pred, average='macro')
          jaccard_score_sum_v1 = jaccard_score_sum_v1 + jaccard_score_v1

          f1_score_v1 = f1_score(y_true, y_pred)
          f1_score_sum_v1 = f1_score_sum_v1 + f1_score_v1

          pixel_accuracy_v1 = pixel_accuracy(y_true, y_pred)
          pixel_accuracy_sum_v1 = pixel_accuracy_sum_v1 + pixel_accuracy_v1

          nsum_v1 = nsum_v1 + 1

          count_fire_pixel_mask = np.sum(y_true)
          count_fire_pixel_pred = np.sum(y_pred)
          
          step += 1
          if step%100 == 0:
              print('Step {} of {}'.format(step, steps)) 
              
      except Exception as e:
          print(e)
          
          with open(os.path.join(OUTPUT_DIR, "error_log.txt"), "a+") as myfile:
              myfile.write(str(e))
      


  y_pred_all_v1 = np.array(y_pred_all_v1, dtype=np.uint8)
  y_pred_all_v1 = y_pred_all_v1.flatten()

  y_true_all_v1 = np.array(y_true_all_v1, dtype=np.uint8)
  y_true_all_v1 = y_true_all_v1.flatten()

  tn, fp, fn, tp = statistics3 (y_true_all_v1, y_pred_all_v1)

  P = float(tp)/(tp + fp)
  R = float(tp)/(tp + fn)
  IoU = float(tp)/(tp+fp+fn)
  F = (2 * P * R)/(P + R)
  print('P: :', P, ' R: ', R, ' IoU: ', IoU, ' F-score: ', F)
  print('Jaccard score average', jaccard_score_sum_v1/nsum_v1)

In [ ]:
with tf.device('/device:GPU:0'):
  getScores()

# Masks: 2849
# Pred.: 2849
Step 100 of 2849
Step 200 of 2849
Step 300 of 2849
Step 400 of 2849
Step 500 of 2849
Step 600 of 2849
Step 700 of 2849
Step 800 of 2849
Step 900 of 2849
Step 1000 of 2849
Step 1100 of 2849
Step 1200 of 2849
Step 1300 of 2849
Step 1400 of 2849
Step 1500 of 2849
Step 1600 of 2849
Step 1700 of 2849
Step 1800 of 2849
Step 1900 of 2849
Step 2000 of 2849
Step 2100 of 2849
Step 2200 of 2849
Step 2300 of 2849
Step 2400 of 2849
Step 2500 of 2849
Step 2600 of 2849
Step 2700 of 2849
Step 2800 of 2849
P: : 0.9303219792597196  R:  0.9911039390930155  IoU:  0.9226176719821277  F-score:  0.9597515776820595
Jaccard score average 0.9662407703618261


### 7. Conclusiones

Se logra una mejora de al menos dos puntos porcentuales en la detección de incendios en imágenes de Sur América en casi todas las métricas excepto el Recall con respecto al modelo pre-entrenado creado en Pereira et. al, tal como muestra la siguiente tabla:

|                      | Precision          | Recall             | Intersection Over Union | F-Score            | Jaccard Score Average |
|----------------------|--------------------|--------------------|-------------------------|--------------------|-----------------------|
| Modelo pre-entrenado | 0.8749501323758749 | 0.9982207878186031 | 0.873587775202781       | 0.9325293287721537 | 0.9463829670347707    |
| Modelo fine-tuning   | 0.9303219792597196 | 0.9911039390930155 | 0.9226176719821277      | 0.9597515776820595 | 0.9662407703618261    |

El valor alto en el recall del modelo original deja entrever que, como es de esperarse, la prioridad del modelo pre-entrenado es detectar todos los píxeles que sean de fuego y no dejar ninguno por fuera más que detectar con precisión un píxel con incendio. Se menciona que es de esperarse ya que en este contexto el no detectar un píxel que en realidad sí tiene un incendio (falso negativo) puede hacer que no se envíen los recursos necesarios para apagar un incendio cuando en realidad sí son necesarios, lo cual es mucho peor que detectar un incendio cuando en realidad no existe (falso positivo).

Dada la orientación del modelo original hacia el recall, era de esperarse que el recall obtenido luego del fine-tuning (siempre y cuando este funcionara bien) fuera prácticamente igual al del modelo original, tal como se observa en la tabla. Los valores de recall de ambos modelos son iguales hasta el segundo decimal, por lo que la diferencia en este valor se puede ver como marginal.

Ahora bien, en cuanto a las demás métricas, la mejora más importante está en la precisión, donde se observa una mejora de 6 puntos porcentuales, seguida del Intersection Over Union (IoU) con 5 puntos porcentuales, y finalmente el F-score y el Jaccard score promedio con 2 puntos porcentuales. La mejora en todas estas métricas permite concluir que el Fine-Tuning del modelo usando imágenes específicas de una región, en este caso Sur América, permite mejorar la detección de incendios sobre esta región en particular, permitiendo especializar el modelo en caso de ser necesario.

Teniendo en cuenta que el modelo evaluado es el generado luego de 3 épocas, es difícil concluir si el modelo generado luego de las 5 épocas, u otro modelo generado luego de más épocas, pudiera tener un rendimiento mejor a este modelo de 3 épocas, o si por el contrario estos modelos de más épocas pudieran estar sobre ajustados y por ende tener un rendimiento más bajo con datos de test. Disponiendo de los recursos necesarios, se podría probar luego de cuál número de épocas se comienza a sobre ajustar el modelo. Por ahora, lo que se puede concluir es que este modelo de fine-tuning de 3 épocas no está sobreajustado dado el buen rendimiento que se tuvo con los datos de test.

Finalmente, hay que decir que trabajos como el de Pereira et. al son muy valiosos para la comunidad científica ya que potencian la investigación. De hecho, durante la investigación del estado del arte de este proyecto se encontraron dos artículos que usaban como base el Dataset de imágenes satelitales anotadas construidas en este trabajo, ambos escritos en 2022, lo cual es un gran avance para la detección de incendios usando imágenes satelitales, y permitirá construir cada vez mejores modelos que permitan, potencialmente, detectar a tiempo los incendios forestales alrededor del mundo y manejarlos adecuadamente, evitando tragedias como los incendios de California o de la Selva Amazónica.